## CNN Text Classification with Keras

In [1]:
import numpy as np
from collections import defaultdict
import re, sys, os, csv, keras
from many_stop_words import get_stop_words
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from keras import regularizers, initializers, optimizers, callbacks
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from keras.layers import Embedding
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding, Merge, Dropout, LSTM, GRU, Bidirectional
from keras.models import Model
from keras import backend as K
from keras.engine.topology import Layer, InputSpec
print("Using Keras version",keras.__version__)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Using Keras version 2.1.4


Read from dataset (`csv` file)

In [2]:
texts, labels = [], []
print("Reading from csv file...")
with open('data.csv') as csvfile:
    readCSV = csv.reader(csvfile, delimiter=',')
    for row in readCSV:
        texts.append(row[0])
        labels.append(row[1])
print("Done!")

Reading from csv file...
Done!


In [3]:
MAX_NB_WORDS = 40000
MAX_SEQUENCE_LENGTH = 30
VALIDATION_SPLIT = 0.2

In [4]:
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))
data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

Found 30542 unique tokens.


In [5]:
labels = to_categorical(np.asarray(labels)) # convert to one-hot encoding vectors
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

Shape of data tensor: (40000, 30)
Shape of label tensor: (40000, 4)


In [6]:
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
nb_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

In [7]:
x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_val = data[-nb_validation_samples:]
y_val = labels[-nb_validation_samples:]

print('Number of entries in each category:')
print("Training:\n",y_train.sum(axis=0))
print("Validation:\n",y_val.sum(axis=0))

Number of entries in each category:
Training:
 [ 7696. 10951. 12182.  1171.]
Validation:
 [1948. 2673. 3117.  262.]


### Preparing the Embedding layer
Compute an index mapping words to known embeddings, by parsing the data dump of pre-trained embeddings: [GloVe](https://nlp.stanford.edu/projects/glove/) vectors from Stanford NLP. For new words, a "randomised vector" will be created.

In [8]:
EMBEDDING_DIM = 200
GLOVE_DIR = "dataset/glove/glove.twitter.27B.200d.txt"
embeddings_index = {}
f = open(GLOVE_DIR)
print("Loading GloVe from:",GLOVE_DIR,"...",end="")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print("Done.\nProceeding with Embedding Matrix...")
embedding_matrix = np.random.random((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
print("Completed!")

Loading GloVe from: dataset/glove/glove.twitter.27B.200d.txt ...Done.
Proceeding with Embedding Matrix...
Completed!


After computing our embedding matrix, load this embedding matrix into an `Embedding` layer. Toggle `trainable=False` to prevent the weights from being updated during training.

In [12]:
embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

### Simplified 1D CNN
[Reference](https://github.com/richliao/textClassifier), [Source](https://blog.keras.io/using-pre-trained-word-embeddings-in-a-keras-model.html) and [Notes](http://www.wildml.com/2015/11/understanding-convolutional-neural-networks-for-nlp/)

In [15]:
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32') # 1000
embedded_sequences = embedding_layer(sequence_input)
l_cov1= Conv1D(128, 3, activation='relu',kernel_regularizer=regularizers.l2(0.01))(embedded_sequences)
l_cov2= Conv1D(128, 3, activation='relu',kernel_regularizer=regularizers.l2(0.01))(l_cov1)
l_pool1 = MaxPooling1D(3)(l_cov2)
#l_drop1 = Dropout(0.2)(l_pool1)
l_cov3 = Conv1D(128, 3, activation='relu',kernel_regularizer=regularizers.l2(0.01))(l_pool1)
l_pool4 = MaxPooling1D(6)(l_cov3) # global max pooling
l_flat = Flatten()(l_pool4)
l_dense = Dense(128, activation='relu')(l_flat)
preds = Dense(4, activation='softmax')(l_dense) # 4 categories

In [17]:
model = Model(sequence_input, preds)
adadelta = optimizers.Adadelta(lr=1.0, rho=0.95, epsilon=None, decay=0)
model.compile(loss='categorical_crossentropy',
              optimizer=adadelta,
              metrics=['acc'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 30)                0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 30, 200)           6108600   
_________________________________________________________________
conv1d_7 (Conv1D)            (None, 28, 128)           76928     
_________________________________________________________________
conv1d_8 (Conv1D)            (None, 26, 128)           49280     
_________________________________________________________________
max_pooling1d_5 (MaxPooling1 (None, 8, 128)            0         
_________________________________________________________________
conv1d_9 (Conv1D)            (None, 6, 128)            49280     
_________________________________________________________________
max_pooling1d_6 (MaxPooling1 (None, 1, 128)            0         
__________

In [23]:
def step_cyclic(epoch):
    try:
        if epoch%11==0:
            return float(2)
        elif epoch%3==0:
            return float(1.1)
        elif epoch%7==0:
            return float(0.6)
        else:
            return float(1.0)
    except:
        return float(1.0)
        
tensorboard = callbacks.TensorBoard(log_dir='./logs', histogram_freq=0, batch_size=256, write_grads=True , write_graph=True)
model_checkpoints = callbacks.ModelCheckpoint("checkpoints", monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period=1)
lr_schedule = callbacks.LearningRateScheduler(step_cyclic)

In [24]:
print("Training Progress:")
model.fit(x_train, y_train, validation_data=(x_val, y_val),
          epochs=200, batch_size=128,
          callbacks=[tensorboard, model_checkpoints, lr_schedule])

Training Progress:
Train on 32000 samples, validate on 8000 samples
Epoch 1/200
32000/32000 [==============================] - 7s 222us/step - loss: 1.1012 - acc: 0.5102 - val_loss: 1.0972 - val_acc: 0.5131
Epoch 2/200
32000/32000 [==============================] - 7s 219us/step - loss: 1.0843 - acc: 0.5211 - val_loss: 1.0851 - val_acc: 0.5151
Epoch 3/200
32000/32000 [==============================] - 7s 228us/step - loss: 1.0840 - acc: 0.5211 - val_loss: 1.0832 - val_acc: 0.5204
Epoch 4/200
32000/32000 [==============================] - 8s 235us/step - loss: 1.0852 - acc: 0.5214 - val_loss: 1.0934 - val_acc: 0.5072
Epoch 5/200
32000/32000 [==============================] - 7s 226us/step - loss: 1.0833 - acc: 0.5223 - val_loss: 1.0882 - val_acc: 0.5156
Epoch 6/200
32000/32000 [==============================] - 7s 231us/step - loss: 1.0834 - acc: 0.5218 - val_loss: 1.0797 - val_acc: 0.5241
Epoch 7/200
32000/32000 [==============================] - 7s 225us/step - loss: 1.0834 - acc: 0.5

32000/32000 [==============================] - 7s 215us/step - loss: 1.0754 - acc: 0.5270 - val_loss: 1.0779 - val_acc: 0.5231
Epoch 118/200
32000/32000 [==============================] - 7s 213us/step - loss: 1.0765 - acc: 0.5250 - val_loss: 1.0785 - val_acc: 0.5186
Epoch 119/200
32000/32000 [==============================] - 7s 213us/step - loss: 1.0761 - acc: 0.5234 - val_loss: 1.0762 - val_acc: 0.5218
Epoch 120/200
32000/32000 [==============================] - 7s 218us/step - loss: 1.0671 - acc: 0.5312 - val_loss: 1.0740 - val_acc: 0.5251
Epoch 121/200
32000/32000 [==============================] - 7s 215us/step - loss: 1.0780 - acc: 0.5219 - val_loss: 1.0767 - val_acc: 0.5204
Epoch 122/200
32000/32000 [==============================] - 7s 216us/step - loss: 1.0933 - acc: 0.5140 - val_loss: 1.0875 - val_acc: 0.5150
Epoch 123/200
32000/32000 [==============================] - 7s 215us/step - loss: 1.0768 - acc: 0.5256 - val_loss: 1.0783 - val_acc: 0.5198
Epoch 124/200
32000/32000 [